In [ ]:
# import required libraries
import pandas as pd
import numpy as np
import geog
import simplekml
import shapely

In [ ]:
# read in dataframe of network data (this is a dummy dataset)
df = pd.read_csv('network_data.csv')
df.info()
# the dataframe contains: 
    # a cell identifier, 
    # cell locations, 
    # the number of connections to a cell, 
    # the direction the cell faces and 
    # the average distance from the cell which users connect to


In [ ]:
# define functions to create different components required to create wedges

# starting with a circle...
def createCircle(lat,lng,n_points,distance_radius_meters):
    p = (lng, lat)
    n_points = n_points
    d = distance_radius_meters  # meters
    angles = np.linspace(0, 360, n_points)
    polygon = geog.propagate(p, angles, d)
    return shapely.geometry.Polygon(polygon)

# ...then two lines to intersect the circle and split it into a wedge and the remainder...
def createLine(lat,lng,n_points,distance_radius_meters,angle):
    p = shapely.geometry.Point([lng, lat])
    n_points = n_points
    d = distance_radius_meters  # meters
    polygon = geog.propagate(p, angle, d, bearing=True)
    try:
        return shapely.geometry.LineString([(lng, lat), (polygon[0], polygon[1])])
    except:
        return None
    
# ...and implement the intersection and pick the wedge by selecting the shape with the minimum area between the two (the wedge and the remainder)
def createWedge(circle, l1, l2):
    if (circle.is_valid) & (l1.is_valid) & (l2.is_valid):
        plist = list(circle.difference(l1.buffer(1e-6)).difference(l2.buffer(1e-6)))
        return plist[[x.area for x in plist].index(min([x.area for x in plist]))]
    else:
        return np.nan